# Day 01 — Prompting styles + simple agent execution

This notebook compares **zero-shot**, **few-shot**, and **role prompting**, then runs them **sequentially** and **in parallel** to show the difference in execution style.

In [ ]:
import asyncio
from pathlib import Path

from openai import OpenAI

client = OpenAI()

PROMPTS_DIR = Path("prompts")

def load_prompt(filename: str) -> str:
    return (PROMPTS_DIR / filename).read_text()

def render_prompt(template: str, question: str) -> str:
    return template.replace("{question}", question)

def call_openai(prompt: str, model: str = "gpt-4o-mini") -> str:
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content

async def call_openai_async(prompt: str, model: str = "gpt-4o-mini") -> str:
    return await asyncio.to_thread(call_openai, prompt, model)


In [ ]:
question = "Give me a 3-step plan to build a small ML project."

templates = {
    "zero_shot": load_prompt("zero_shot.txt"),
    "few_shot": load_prompt("few_shot.txt"),
    "role_prompt": load_prompt("role_prompt.txt"),
}

rendered_prompts = {
    name: render_prompt(template, question)
    for name, template in templates.items()
}
rendered_prompts

## Sequential execution
Each prompt is run one after another. This is the simplest agent pattern.

In [ ]:
sequential_results = {}
for name, prompt in rendered_prompts.items():
    sequential_results[name] = call_openai(prompt)

sequential_results

## Parallel execution
The prompts are sent concurrently. This is useful when tasks are independent.

In [ ]:
async def run_parallel(prompts: dict) -> dict:
    tasks = {
        name: asyncio.create_task(call_openai_async(prompt))
        for name, prompt in prompts.items()
    }
    return {name: await task for name, task in tasks.items()}

parallel_results = await run_parallel(rendered_prompts)
parallel_results

## Quick comparison
I eyeball these outputs to see how different prompts shape the answers.

In [ ]:
for name, output in parallel_results.items():
    print(f"\n--- {name} ---\n{output}")